In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from math import sqrt

import pandas as pd
import numpy as np
import os
from pathlib import Path
from IPython.display import display, FileLink
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# House prices predictions: random forest

For more information about the feature engineering in this notebook, see [./linear-regression.ipynb](Linear Regression walkthrough).

## 1. Download dataset

In [3]:
PATH = Path('./data')
PATH.mkdir(exist_ok=True)

In [5]:
!kaggle competitions download -c house-prices-advanced-regression-techniques -p {PATH}

data_description.txt: Downloaded 13KB of 13KB to data
train.csv.gz: Downloaded 89KB of 89KB to data
train.csv: Downloaded 450KB of 450KB to data
test.csv.gz: Downloaded 82KB of 82KB to data
test.csv: Downloaded 441KB of 441KB to data
sample_submission.csv.gz: Downloaded 15KB of 15KB to data
sample_submission.csv: Downloaded 31KB of 31KB to data


## 2. Prepare dataset

In [12]:
df_raw = pd.read_csv(PATH / 'train.csv')

### Extract and prepare target values

In [13]:
sale_price = df_raw.pop('SalePrice')
sale_price_log = np.log(sale_price)

### Find columns

In [14]:
house_ids = df_raw.pop('Id')

In [15]:
MAX_N_UNIQUE = 50

continuous_columns = set([
    col_name for col_name, col in df_raw.items()
    if len(col.unique()) > MAX_N_UNIQUE])

In [16]:
continuous_columns = list(continuous_columns | set([
    'LowQualFinSF', 'BsmtHalfBath', 'BsmtFullBath', 'FullBath',
    'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', '3SsnPorch',
    'PoolArea', 'MiscVal', 'YrSold', 'Fireplaces']))

In [17]:
categorical_columns = [col for col in df_raw.columns if col not in continuous_columns]

In [18]:
assert len(df_raw.columns) == len(categorical_columns + continuous_columns)

### Prepare categorical

In [19]:
for col_name, col in df_raw[categorical_columns].items():
    df_raw[col_name] = col.astype('category').cat.as_ordered()

In [20]:
# Quality measures (Excellent, Good, Average, Fair, Poor)
df_raw.ExterQual.cat.set_categories(['Ex', 'Gd', 'TA', 'Fa', 'Po'], ordered=True, inplace=True)
df_raw.ExterCond.cat.set_categories(['Ex', 'Gd', 'TA', 'Fa', 'Po'], ordered=True, inplace=True)
df_raw.BsmtQual.cat.set_categories(['Ex', 'Gd', 'TA', 'Fa', 'Po'], ordered=True, inplace=True)
df_raw.BsmtExposure.cat.set_categories(['Ex', 'Gd', 'TA', 'Fa', 'Po'], ordered=True, inplace=True)
df_raw.BsmtFinType1.cat.set_categories(['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf'], ordered=True, inplace=True)
df_raw.BsmtFinType2.cat.set_categories(['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf'], ordered=True, inplace=True)
df_raw.HeatingQC.cat.set_categories(['Ex', 'Gd', 'TA', 'Fa', 'Po'], ordered=True, inplace=True)
df_raw.KitchenQual.cat.set_categories(['Ex', 'Gd', 'TA', 'Fa', 'Po'], ordered=True, inplace=True)
df_raw.FireplaceQu.cat.set_categories(['Ex', 'Gd', 'TA', 'Fa', 'Po'], ordered=True, inplace=True)
df_raw.GarageFinish.cat.set_categories(['Fin', 'Rfn', 'Unf'], ordered=True, inplace=True)
df_raw.GarageQual.cat.set_categories(['Ex', 'Gd', 'TA', 'Fa', 'Po'], ordered=True, inplace=True)
df_raw.GarageCond.cat.set_categories(['Ex', 'Gd', 'TA', 'Fa', 'Po'], ordered=True, inplace=True)
df_raw.PoolQC.cat.set_categories(['Ex', 'Gd', 'TA', 'Fa'], ordered=True, inplace=True)

### Prepare continuous

In [21]:
nas = {}

for col in continuous_columns:
    if not pd.isna(df_raw[col]).sum():
        continue
        
    median = df_raw[col].median()
        
    df_raw[f'{col}_na'] = pd.isna(df_raw[col])
    df_raw[col] = df_raw[col].fillna(median)
    
    nas[col] = median

In [22]:
nas

{'MasVnrArea': 0.0, 'LotFrontage': 69.0, 'GarageYrBlt': 1980.0}

In [23]:
df_raw['LotArea'].head()

0     8450
1     9600
2    11250
3     9550
4    14260
Name: LotArea, dtype: int64

In [24]:
scaler = StandardScaler()
df_raw[continuous_columns] = scaler.fit_transform(df_raw[continuous_columns])

In [25]:
df_raw['LotArea'].head()

0   -0.207142
1   -0.091886
2    0.073480
3   -0.096897
4    0.375148
Name: LotArea, dtype: float64

### Numericalise

In [26]:
df_numeric = df_raw.copy()

for col_name in categorical_columns:
    # Use +1 to push the -1 NaN value to 0
    df_numeric[col_name] = df_numeric[col_name].cat.codes + 1

### Create validation set

In [28]:
X_train, X_val, y_train, y_val = train_test_split(df_numeric.values, sale_price_log, test_size=0.2, random_state=42)

## 3. Train evaluate model

In [29]:
model = RandomForestRegressor()

In [30]:
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

### Basic evaluation

In [34]:
print(model.score(X_train, y_train))
print(model.score(X_val, y_val))

0.9715990984356334
0.8688624184424235


In [35]:
preds = model.predict(X_val)

In [36]:
rms = sqrt(((y_val - preds) ** 2).mean())
print(f'RMSE: {rms}')

RMSE: 0.15643543546453792


## 4. Submit predictions

We now need to prepare the test set using exactly the same preparation we used with the training data.

In [37]:
df_test_raw = pd.read_csv(PATH / 'test.csv')

In [38]:
house_ids = df_test_raw.pop('Id')

In [39]:
for col_name in categorical_columns:
    df_test_raw[col_name] = (
        pd.Categorical(df_test_raw[col_name], categories=df_raw[col_name].cat.categories, ordered=True))

In [40]:
for col in continuous_columns:
    if col not in nas:
        continue

    df_test_raw[f'{col}_na'] = pd.isna(df_test_raw[col])
    df_test_raw[col] = df_test_raw[col].fillna(nas[col])

In [41]:
# Handle any other nas
df_test_raw[continuous_columns] = df_test_raw[continuous_columns].fillna(df_test_raw[continuous_columns].median())

In [42]:
df_test_raw[continuous_columns] = scaler.transform(df_test_raw[continuous_columns])

In [43]:
df_test = df_test_raw.copy()

for col_name in categorical_columns:
    # Use +1 to push the -1 NaN value to 0
    df_test[col_name] = df_test[col_name].cat.codes + 1

In [44]:
test_preds = model.predict(df_test)

In [45]:
pd.DataFrame({'Id': house_ids, 'SalePrice': np.exp(test_preds)}).to_csv(f'{PATH}/sub_rf.csv', index=False)

In [46]:
FileLink(PATH / 'sub_rf.csv')

/Users/lex/code/notebooks/kaggle/house-prices-advanced-regression-techniques/data/sub_rf.csv

Does a little worse than the Linear Model with a score of 0.15686.